In [ ]:
# ===============================
# MASTER PIPELINE – EGYÉNI LSTM
# multi-input LSTM, mert különféle információforrásokat (idősorokat) kezel párhuzamosan, majd a végén összefűzi őket
# ===============================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping


# ---------------------------------------------------
# 0) Segédfüggvények
# ---------------------------------------------------
def topk_acc_numpy(y_true, y_proba, k=5):
    # Ellenőrzi, hogy y_proba tényleg kétdimenziós-e
    if y_proba.ndim != 2:
        raise ValueError("y_proba 2D mátrix legyen (n_samples, n_classes).")
    
    # a lehetséges cellákra (vagyis célosztályokra) vonatkozó valószínűségek mátrixa -> oszlopainak száma = lehetséges osztályok száma
    n_classes = y_proba.shape[1]
    if n_classes == 0:
        return 0.0
    
    # megkeresi a k darab legvalószínűbb osztály indexét minden egyes mintára ->
    # majd ellenőrzi, hogy a valódi címke ezek között van-e ->
    # végül kiszámolja a helyes találatok arányát
    k_eff = min(k, n_classes)
    topk_idx = np.argpartition(y_proba, -k_eff, axis=1)[:, -k_eff:]
    correct = (topk_idx == y_true[:, None]).any(axis=1)
    return float(correct.mean())

def train_test_split_seq(X_list, y, split_ratio=0.8):
    # ez a függvény azért kell, mert több bemeneti típusú mátrixunk van (pl. cellák, félórák, numerikus jellemzők) -> a szokásos train_test_split() csak egy mátrixot tud kezelni
    # mivel midegyikben ugyanannyi sor van, ezért elég ugyanott elvágni őket
    n = len(y)  # szekvenciák száma
    split = int(n * split_ratio)  # kiszámolja, hány szekvencia kerül a tanító részbe, avagy a vágáspont indexét
    
    # az összes bemeneti mátrixot elvágja ugyanott
    X_train = [X[:split] for X in X_list]
    X_test  = [X[split:] for X in X_list]
    # az output mátrixot is elvágja
    y_train, y_test = y[:split], y[split:]

    return X_train, X_test, y_train, y_test

# ---------------------------------------------------
# 1) Előkészítés
# ---------------------------------------------------
def preprocess_dataframe(df, coarsen_factor=2):
    df = df.copy()

    # koordináták durvítása -> 200 cellából 100 cella -> majd cell_id képzése 
    # df["x_coarse"] = (df["x"] // coarsen_factor).astype(int)
    # df["y_coarse"] = (df["y"] // coarsen_factor).astype(int)
    # df["cell_id"] = df["x_coarse"] * 1000 + df["y_coarse"] # egyedi cellaazonosító képzése

    # időjellemzők képzése
    #df["hour"] = df["date"].dt.hour  ez hibás
    df["hour"] = df["t"].astype(int)
    # órák ciklikusságának kezelése sin és cos-sal
    df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 48)
    df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 48)
    df["is_weekend"] = (df["date"].dt.dayofweek >= 5).astype(int)

    # skálázandó numerikus jellemzők
    num_cols = ["temperature", "rain", "daily_radius_of_gyration",
                "radius_of_gyration", "unique_cells_count"]
    # hiányzó értékek 0-val való pótlása (kevés van ezért megengedett)
    for c in num_cols:
        if c not in df.columns:
            df[c] = 0.0

    # itt még ne skálázzuk, csak térjünk vissza az oszlopokkal (mert nem felhasználónként kell skálázni)
    df[num_cols] = df[num_cols].astype(float)

    # ha talál végtelen értéket, lecseréli NaN-ra, majd azokat 0-val pótolja
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)
    return df

# ---------------------------------------------------
# 2) Szekvenciák építése LSTM-hez
# ---------------------------------------------------
def build_sequences_for_user(user_df, seq_len=20):
    df = user_df.copy().reset_index(drop=True)

    # CELL_ID KÓDOLÁSA AZ LSTM-NEK MEGFELELŐ MÓDON
    # OUTPUT célérték képzése: a következő sor cell_id-ja
    df["target"] = df["cell_id"].shift(-1)
    df = df.dropna(subset=["target"]).reset_index(drop=True)  # az utolsó sor kiesik, mert nincs következő sor

    # cella_id (kategorikus) kódolása egész számokra (osztályokra) -> ne értelmezzen matematikai kapcsolatot a cell_id-k között
    # softmax miatt is kell (osztályokat jósol)
    # y_all: az összes célérték kódolt változata
    # uniques: az egyedi célértékek eredeti értékei (target-k)
    y_all, uniques = pd.factorize(df["target"].astype(int))
    df["target_enc"] = y_all.astype(np.int32)
    num_classes = len(uniques) # az egyedi osztályok száma (max 10)

    # ugyanaz mint feljebb, csak a bemeneti cell_id-kra
    cell_all, cell_uniques = pd.factorize(df["cell_id"].astype(int))
    df["cell_enc"] = cell_all.astype(np.int32)
    cell_vocab_size = int(df["cell_enc"].max()) + 2 # +2 az indexelés miatt és a padding miatt (ez megy az embedding réteg input_dim-jébe)

    df["dayofweek_enc"], _ = pd.factorize(df["days_of_week"])

    #hour_ids = df["hour"].astype(np.int32).values  # numpy tömbbé alakitása az órák értékeinek
    #num_feats = df[["hour_sin", "hour_cos", "is_weekend"]].astype(np.float32).values
    # MINDEN JELLEMZŐ BEVONÁSA
    hour_ids = df["hour"].astype(np.int32).values
    num_cols = [
        "temperature", "rain", "daily_radius_of_gyration",
        "radius_of_gyration", "unique_cells_count", "fraction_missing",
        "hour_sin", "hour_cos", "is_weekend",
        "is_home", "is_workplace",
        "dayofweek_enc"
    ]
    num_feats = df[num_cols].astype(np.float32).values

    # szekvenciák építése az adott user összes sorából
    X_cell, X_hour, X_num, y = [], [], [], []
    for i in range(len(df) - seq_len):
        sl = slice(i, i + seq_len)
        X_cell.append(df["cell_enc"].values[sl])
        X_hour.append(hour_ids[sl])
        X_num.append(num_feats[sl, :])
        y.append(df["target_enc"].values[i + seq_len - 1])

    # ha nincs elég adat szekvenciák képzéséhez, visszatér None-nal pl. ha 20-nál kevesebb rekord volt a userre 
    if len(y) == 0:
        return None

    # numpy tömbökké alakítás és megfelelő típusok beállítása az LSTM-hez
    return (np.asarray(X_cell, dtype=np.int32),
            np.asarray(X_hour, dtype=np.int32),
            np.asarray(X_num,  dtype=np.float32),
            np.asarray(y,      dtype=np.int32),
            cell_vocab_size, 48, num_classes)  # 48 mert 0-47 órák

# ---------------------------------------------------
# 3) Modell
# ---------------------------------------------------
def build_lstm_multi(num_cells, num_hours, num_numeric, num_classes):

    # cella-szekvencia bemenet - 1.tipus
    inp_cell = layers.Input(shape=(None,), dtype="int32", name="cell_in")
    # embedding réteg a cellaazonosítókhoz (64 dimenziós vektorok, mert nincs folytonos kapcsolat a cella id-k között, a modell maga tanulja meg a cellák közötti kapcsolatokat)
    # hasonlóan működik, mint a szavaknál a NLP-ben
    emb_cell = layers.Embedding(num_cells, 64, mask_zero=False)(inp_cell)
    x_cell = layers.LSTM(128, recurrent_dropout=0.1)(emb_cell)  # Ez az LSTM 128 neuronnal végigmegy a cellaembedding szekvencián és megtanulja, milyen sorrendek jellemzők a mozgásra -> 128 mintát tud párhuzamosan megfigyelni és megjegyezni

    # idő-szekvencia bemenet - 2.tipus
    inp_hour = layers.Input(shape=(None,), dtype="int32", name="hour_in")
    emb_hour = layers.Embedding(num_hours, 16, mask_zero=False)(inp_hour)
    x_hour = layers.LSTM(32)(emb_hour)  # megtanulja pl. éjszakai - nappali mozgásminták különbözőségét, rush-hour stb.

    # numerikus jellemzők szekvencia bemenet - 3.tipus (pl. eső, hőmérséklet, fraction_missing, is_home, is_workplace stb.)
    inp_num = layers.Input(shape=(None, num_numeric), dtype="float32", name="num_in")
    x_num = layers.LSTM(32)(inp_num)

    # összefűzi a 3 tipus kimeneteit: (batch_size, 128 + 32 + 32) = (batch_size, 192)
    x = layers.Concatenate()([x_cell, x_hour, x_num])
    # Egy rejtett réteg, amely a három forrásból származó információt kombinálja -> kb fele annyi neuronnal, mint az összefűzött bemenet, aktiváció ReLU -> nemlineáris kapcsolatokat is tud tanulni
    x = layers.Dense(128, activation="relu")(x)
    # kimeneti réteg softmax aktivációval -> célosztályok (célcellák) valószínűségei
    #x = layers.Dropout(0.2)(x)
    #x = layers.BatchNormalization()(x)  -> tulzottan lassította a tanulást
    out = layers.Dense(num_classes, activation="softmax")(x)

    # multi-LSTM modell összeállítása
    model = models.Model(inputs=[inp_cell, inp_hour, inp_num], outputs=out)
    # modell beállítások: Adam (Adaptive Moment Estimation) optimalizáló -> automatikusan állítja a tanulási rátát, gyorsan konvergál
    # sparse categorical crossentropy veszteségfüggvény -> mert a célértékek (y) egész számokként vannak kódolva és nem one-hot kódoltak
    # metrics: accuracy -> ez határozza meg, mit figyel a Keras a tanulás során és irja ki az epoch végén
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

def run_user_model_lstm_multi(df, uid, seq_len=20, epochs=20):
    
    # adott user adatainak kiválasztása (ha elég adat van róla)
    dfu = df[df["uid"] == uid].copy()
    if len(dfu) < 50:
        return None

    # kiválasztott user szekvenciáinak felépítése
    seqs = build_sequences_for_user(dfu, seq_len=seq_len)
    if seqs is None:
        return None
    X_cell, X_hour, X_num, y, cell_vocab, hour_vocab, num_classes = seqs

    # adatok felosztása tanító és teszt részre
    X_train, X_test, y_train, y_test = train_test_split_seq([X_cell, X_hour, X_num], y, split_ratio=0.8)

    # modell felépítése adott userre szabva
    model = build_lstm_multi(cell_vocab, hour_vocab, X_num.shape[2], num_classes) # X_num.shape[2] -> numerikus jellemzők száma időpillanatonként

    # korai leállítás beállítása -> ha 3 egymást követő epochban nem javul a validációs veszteség, akkor megállítja a tanulást és visszaállítja a legjobb súlyokat
    early_stop = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

    # modell tanítása
    history = model.fit(
                X_train, y_train,
                validation_split=0.1,
                epochs=epochs,
                batch_size=64,
                verbose=0,
                callbacks=[early_stop]
                )

    # --- Tanulási görbék vizualizálása ---

    # plt.figure(figsize=(8,5))
    # plt.plot(history.history["loss"], label="Train loss")
    # plt.plot(history.history["val_loss"], label="Validation loss")
    # plt.xlabel("Epoch")
    # plt.ylabel("Loss")
    # plt.legend()
    # plt.title(f"User {uid} - Loss görbe")
    # plt.show()

    # plt.figure(figsize=(8,4))
    # plt.plot(history.history["accuracy"], label="Train acc")
    # plt.plot(history.history["val_accuracy"], label="Val acc")
    # plt.xlabel("Epoch")
    # plt.ylabel("Pontosság")
    # plt.legend()
    # plt.title(f"User {uid} - Accuracy görbe")
    # plt.show() 

    # --- Kiértékelés külön a teszten ---
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"[{uid}] Test accuracy: {test_acc:.3f}, Test loss: {test_loss:.3f}")

    proba_test = model.predict(X_test, verbose=0)
    pred_test = np.argmax(proba_test, axis=1)
    acc = accuracy_score(y_test, pred_test)
    top5 = topk_acc_numpy(y_test, proba_test, k=5)

    return {
        "uid": uid,
        "model_type": "LSTM-multi", # modell típusa
        "num_classes": num_classes, # célosztályok száma
        "train_accuracy": history.history["accuracy"][-1],  # tanuló halmazon mért pontosság
        "val_accuracy": history.history["val_accuracy"][-1], # validációs halmazon mért pontosság
        "test_accuracy": test_acc,                     # teszt halmazon mért pontosság
        "accuracy": acc,                              #  pontosság (hány helyes predikció a teszten)
        "top5_accuracy": top5,                       # top-5 pontosság (hány esetben volt benne a helyes címke a 5 legvalószínűbb predikció között)
        "epochs_ran": len(history.history["loss"]),          # hány epoch futott le ténylegesen
        "final_train_loss": history.history["loss"][-1],     # utolsó tanító loss
        "final_val_loss": history.history["val_loss"][-1],   # utolsó validációs loss
        "best_val_loss": min(history.history["val_loss"]),   # legjobb validációs loss
        "best_epoch": np.argmin(history.history["val_loss"]) + 1,  # melyik epochban volt a legjobb
        "user_samples": len(dfu)  # hány sor volt az adott userhez az eredeti adatokban
    }


# ---------------------------------------------------
# 5) Futtatás checkpointtal
# ---------------------------------------------------
df = pd.read_csv("df_test_10k.csv", parse_dates=["date"])
df_processed = preprocess_dataframe(df)

# Globális skálázás a numerikus oszlopokra
num_cols = ["temperature", "rain", "daily_radius_of_gyration",
            "radius_of_gyration", "unique_cells_count"]

scaler = StandardScaler()
df_processed[num_cols] = scaler.fit_transform(df_processed[num_cols])

user_ids = sorted(df_processed["uid"].unique())

results_file = "results_checkpoint.csv"
checkpoint_file = "checkpoint.txt"

# ha van korábbi checkpoint, onnan folytatjuk
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        start_index = int(f.read().strip())
else:
    start_index = 0

if os.path.exists(results_file):
    results_df = pd.read_csv(results_file)
    results = results_df.to_dict(orient="records")
else:
    results = []

for i, uid in enumerate(user_ids[start_index:], start=start_index):
    print(f"[{i+1}/{len(user_ids)}] User {uid} fut (LSTM multi-input)...")
    res = run_user_model_lstm_multi(df_processed, uid, seq_len=20, epochs=20)
    if res is not None:
        results.append(res)
        pd.DataFrame(results).to_csv(results_file, index=False)
    # checkpoint frissítése
    with open(checkpoint_file, "w") as f:
        f.write(str(i+1))

print("✅ Kész! Eredmények a fájlban:", results_file)

[9367/10000] User 9366 fut (LSTM multi-input)...
[9366] Test accuracy: 0.378, Test loss: 3.389
[9368/10000] User 9367 fut (LSTM multi-input)...
[9367] Test accuracy: 0.118, Test loss: 3.279
[9369/10000] User 9368 fut (LSTM multi-input)...
[9368] Test accuracy: 0.316, Test loss: 3.877
[9370/10000] User 9369 fut (LSTM multi-input)...
[9369] Test accuracy: 0.333, Test loss: 2.431
[9371/10000] User 9370 fut (LSTM multi-input)...
[9370] Test accuracy: 0.551, Test loss: 2.068
[9372/10000] User 9371 fut (LSTM multi-input)...
[9371] Test accuracy: 0.184, Test loss: 4.980
[9373/10000] User 9372 fut (LSTM multi-input)...
[9372] Test accuracy: 0.400, Test loss: 2.579
[9374/10000] User 9373 fut (LSTM multi-input)...
[9373] Test accuracy: 0.142, Test loss: 4.463
[9375/10000] User 9374 fut (LSTM multi-input)...
[9374] Test accuracy: 0.276, Test loss: 2.013
[9376/10000] User 9375 fut (LSTM multi-input)...
[9375] Test accuracy: 0.358, Test loss: 3.350
[9377/10000] User 9376 fut (LSTM multi-input)...
[